In [8]:
import pandas as pd
import random
import itertools
import numpy as np
import time 
import pickle
from scipy.stats import ttest_ind, chi2_contingency, fisher_exact, ranksums

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Context and Background

In pancreatic cancer, there are two main subtypes that have been identified: 'basal-like' and 'classical.' However, the problem is that we don't really know which gene to target in 'basal-like' subtypes and which to target in 'classical' subtypes so they're functionally useless because we don't know how to treat each uniquely. Therefore, I think it could be interesting to perform a correlation analysis between each of the genes in our lists (both the FDA and also the novel drug list) and see if any of these genes associate strongly with basal-like or classical signature strength.

So, more precisely, we are assessing correlation between GENE X's expression values and signature scores for basal-like/classical across all cells that contain non-zero values for each. Do this only on tumor cells, we don't care about the other cell types. The non-zero expression value of gene X should be pretty straightforward; this is already present in the single cell matrix. To determine the signature score, we will use the same formula as before: 

**Equation:**
Signature Mean - Background Mean = Signature Score 

For each cell, there will be a unique Signature Mean and a unique Background Mean, and therefore a unique Signature Score for each cell.

Calculating Signature Mean:
Extract the specific columns that line up with the gene names in the signature (see each attachment separately).
Take the average value of all of those column values for that one cell/row. You do not take the average of values down a column in this case.

Use the lists attached for basal-like (Moffitt basal-like) and classical like (Moffitt classical-like) to calculate a basal-like signature mean and also a classical signature mean for each cell (you have already computed a background score in the past, so you can just use that same unique value from before for each cell). Then subtract the background mean from the signature mean to get the signature score for each cell. Then similar to the analysis in Part #4, let's generate a list of genes that have the strongest correlations with basal-like scores and then separately also a list that correlates with classical signature scores (using both pearson and spearman, p and r). 

In [1]:
%cd drive/My\ Drive/Broad

/content/drive/My Drive/Broad


In [2]:
%pwd

'/content/drive/My Drive/Broad'

In [3]:
subset = 'fda'
gene_file = '110920_fda drug list v1.csv'

In [9]:
column_names = pd.read_csv('data/columns for X.csv',header = None)
column_names = list(column_names[0])

In [10]:
rows = pd.read_csv('data/rows for X.csv',header = None)
cell_type = list(rows[0])
cell_type = [val.lower() for val in cell_type]

In [11]:
skip_rows = [idx  for idx, val in enumerate(cell_type) if val != 'tumor']

In [12]:
gene_list = pd.read_csv('data/{}'.format(gene_file), header = None)
gene_list = list(gene_list[0])

In [41]:
len(gene_list)

61

In [ ]:
df_gene = pd.read_csv('data/X.csv',names = column_names, skiprows = skip_rows, usecols = lambda x: x in gene_list)

In [38]:
len(skip_rows)

44214

In [ ]:
def get_background(row):
    mean_list = []
    n_cols = len(row)    
    for i in range(25):
        col_rand = random.sample(range(0, n_cols), 50)
        row_rand = row.iloc[col_rand]
        mean_list.append(np.nanmean(row_rand))
        
    return np.nanmean(mean_list)

In [ ]:
chunksize = 3000
background_list = []
start = time.time()
for idx, chunk in enumerate(pd.read_csv('data/X.csv',names = column_names, chunksize=chunksize)):
    background_list.append(chunk.apply(get_background,axis = 1))
    print('completed with {} at {:.2f}'.format(idx,time.time()-start))


In [ ]:
background_arr = pd.concat(background_list,axis = 0)

In [27]:
df_background = pd.DataFrame({'value':background_arr})

In [28]:
df_background.to_csv('outputs/background_values.csv', index = False)

In [13]:
groups_list = [
    ('basal','data/Moffitt Basal-like.csv'),
    ('classifical','data/Moffit Classical-like.csv'),
]

In [45]:

df_basal_list = pd.read_csv('data/Moffitt Basal-like.csv', header = None)
basal_gene_list = list(df_basal_list[0])

df_gene_basal = pd.read_csv('data/X.csv', names = column_names, usecols = lambda x: x in basal_gene_list)
df_gene_basal['score'] = df_gene_basal.mean(axis = 1) - df_background['value'].values
df_gene_basal['cell_type'] = cell_type

df_gene_basal_tumor = df_gene_basal[df_gene_basal['cell_type'] == 'tumor']
basal_score_tumor  = df_gene_basal_tumor['score']

In [ ]:
df_classical_list = pd.read_csv('data/Moffit Classical-like.csv', header = None)
classical_gene_list = list(df_classical_list[0])

df_gene_classical = pd.read_csv('data/X.csv', names = column_names, usecols = lambda x: x in classical_gene_list)
df_gene_classical['score'] = df_gene_classical.mean(axis = 1) - df_background['value'].values
df_gene_classical['cell_type'] = cell_type

df_gene_classical_tumor = df_gene_classical[df_gene_classical['cell_type'] == 'tumor']
classical_score_tumor  = df_gene_classical_tumor['score']

In [ ]:
df_gene['basal_score'] = basal_score_tumor
df_gene['classical_score'] = classical_score_tumor


In [43]:
for gene_in 

,EPHA2,PLA2G2A,CD52,TACSTD2,S1PR1,ATP1A1,ATP1B1,IL1R1,SCTR,CXCR4,RAF1,ATP1B3,TNK2,PDGFRA,FGA,SLC25A4,HDAC3,CSF1R,PDGFRB,GABRP,FGFR4,DDR1,SLC29A1,FYN,NPC1L1,EPHB4,MET,CFTR,HDAC6,IL2RG,DPYSL2,GSR,FGFR1,LYN,CA2,SYK,KCNQ1,FXYD2,CD3E,ADRA2A,FGFR2,SCNN1A,ERBB3,TGFB3,MAP2K1,DPEP1,ERBB2,ACE,YES1,FKBP1A,HCK,SRC,PTK6,EEF2,MAP2K2,INSR,TGFB1,KCNN4,COMT,IFNAR2,ITGB2
0,0.000000,0.0,0.0,2.390654,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.390654,0.0,0.0,0.0,0.000000,0.0,0.0,2.390654,0.000000,2.390654,0.0,3.426277,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.535909,0.0,2.535909,0.0,0.0,0.0
2,0.000000,0.0,0.0,2.274803,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.274803,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.274803,0.0,0.0,0.0,0.0,2.274803,0.0,0.0,2.274803,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,2.274803,0.000000,0.0,0.000000,0.0,2.274803,0.0,0.0,0.0
3,1.918477,0.0,0.0,1.918477,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.918477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.535374,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.918477,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,1.751662,0.0,0.0,1.751662,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.751662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.751662,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.751662,0.0,0.000000,0.0,0.0,0.0


In [44]:
df_gene.mean(axis = 1)

0        0.212933
1        0.083145
2        0.261043
3        0.167365
4        0.143579
           ...   
43812    0.393958
43813    0.277922
43814    0.224782
43815    0.338442
43816    0.333135
Length: 43817, dtype: float64